In [1]:
# Import the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Define settings
import os

pd.set_option("display.precision", 3)
data_dir_path = os.getcwd() + '/dataverse_files/'

In [3]:
# Import data
import pyreadr

croissance_et_climat_decadaires = pyreadr.read_r(data_dir_path + "croissance_et_climat_decadaires.rds" ).popitem()[1]
valorisation_annuelle = pyreadr.read_r(data_dir_path + "valorisation_annuelle.rds" ).popitem()[1]

In [7]:
valuable_columns = ["ucs", "safran", "sol", "gestion", "annee", "decade", "Tmin", "Tmax", "Tmoy", "Rain", "RG", "im", "croissance"]
df = croissance_et_climat_decadaires.loc[:, valuable_columns]

df.dropna(inplace= True)
data_preserved = (df.isna() | df.isnull()).sum().sum() == 0
data_preserved = data_preserved & (df.groupby(["ucs", "safran", "sol", "gestion", "annee"]).count()["decade"] % 37 == 0).all()
assert data_preserved == True

df["annee"] = df["annee"].astype("int64")
df["decade"] = df["decade"].astype("int64")

croissance_et_climat_decadaires_preprocessed = df

In [5]:
valuable_columns = ["ucs", "safran", "sol", "gestion", "annee", "cumul_croissance"]
df = valorisation_annuelle.loc[:, valuable_columns]

df = df.reset_index(drop=True)
df[["annee"]] = df[["annee"]].astype("int64")

valorisation_annuelle_preprocessed = df

In [6]:
valorisation_annuelle_preprocessed

,ucs,safran,sol,gestion,annee,cumul_croissance
0,1005,2245,330562,15,1985,8.757
1,1005,2245,330562,15,1986,14.620
2,1005,2245,330562,15,1987,16.287
3,1005,2245,330562,15,1989,9.063
4,1005,2245,330562,15,1990,11.758
...,...,...,...,...,...,...
535401,998,2511,330563,7,2009,12.433
535402,998,2511,330563,7,2010,9.489
535403,998,2511,330563,7,2011,14.803
535404,998,2511,330563,7,2012,18.890
